In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.datasets import load_iris

In [ ]:
matrix_m = np.matrix(np.random.random((20, 5)))
matrix_p = np.matrix(np.random.random((5, 2)))

matrix_p * matrix_m

In [ ]:
features, target = load_iris(return_X_y=True, as_frame=True)
features.head()

Удалим из стратифицированной выборки треть данных

In [ ]:
kfold = StratifiedKFold(n_splits=3, shuffle=False)
for ((train_index, test_index), index) in zip(kfold.split(features, target), range(0, 5)):
    if index == 1:
        features.loc[test_index] = np.nan

features.isna().sum()

1. Обучить модель по бейзлайну, считать такую модель эталонной
2. Удалить часть данных из стратифицированной выборки
3. Восстановить данные методом ручной группировки и проверить точность модели из бейзлайна
4. Восстановить с помощью IterativeImputer и проверить точность модели из бейзлайна

1. Обучить модель по бейзлайну, считать такую модель эталонной
2. Удалить часть данных из стратифицированной выборки
3. Восстановить данные методом ручной группировки и проверить точность модели из бейзлайна
4. Восстановить с помощью IterativeImputer и проверить точность модели из бейзлайна